##### import libs

In [1]:
from sansa import SANSAConfig

/home/sokolov/workspace/vk_recsys/vk_env/lib/python3.10/site-packages/numba/core/errors.py:194: UserWarning: Insufficiently recent colorama version found. Numba requires colorama >= 0.3.9
  warnings.warn(msg)


In [2]:
import numpy as np
from scipy.sparse import coo_matrix, csr_matrix
from sklearn.model_selection import train_test_split

INFO:numexpr.utils:NumExpr defaulting to 16 threads.


In [3]:
from utils import evaluate, load_data

##### read data

In [4]:
user_item_data, user_meta_data, item_meta_data, test_pairs_data = load_data()

##### split data

In [5]:
ui_train, ui_val = train_test_split(user_item_data,
                                    test_size=0.15,
                                    random_state=42,
                                    shuffle=False)

In [6]:
u_train = ui_train.user_id
i_train = ui_train.item_id
likes_train = ui_train.like
dislikes_train = ui_train.dislike

u_val = ui_val.user_id
i_val = ui_val.item_id
likes_val = ui_val.like
dislikes_val = ui_val.dislike

In [7]:
sparse_train = coo_matrix((likes_train-dislikes_train, (u_train, i_train)))
sparse_val = coo_matrix((likes_val-dislikes_val, (u_val, i_val)))

In [8]:
sparse_train = coo_matrix((likes_train, (u_train, i_train)))
sparse_val = coo_matrix((likes_val, (u_val, i_val)))

##### SANSA model

In [9]:
from sansa import ICFGramianFactorizerConfig

factorizer_config = ICFGramianFactorizerConfig(
    factorization_shift_step = 1e-3,  # initial diagonal shift if incomplete factorization fails
    factorization_shift_multiplier = 2.0,  # multiplier for the shift for subsequent attempts
)

In [10]:
from sansa import UMRUnitLowerTriangleInverterConfig

inverter_config = UMRUnitLowerTriangleInverterConfig(
    scans=1,  # number of scans through all columns of the matrix
    finetune_steps=5,  # number of finetuning steps, targeting worst columns
)

In [11]:
config = SANSAConfig(
    l2 = 20.0,  # regularization strength
    weight_matrix_density = 5e-5,  # desired density of weights
    gramian_factorizer_config = factorizer_config,  # factorizer configuration
    lower_triangle_inverter_config = inverter_config,  # inverter configuration
)

In [12]:
from sansa import SANSA

# Instantiate model with the config
model = SANSA(config)

In [13]:
sparse_train

<183404x337727 sparse matrix of type '<class 'numpy.int16'>'
	with 123817189 stored elements in COOrdinate format>

In [14]:
# Train model on the user-item matrix
model.fit(sparse_train)

INFO:sansa.model:Computing column norms of X^TX...


MemoryError: Unable to allocate 123. GiB for an array with shape (16572477719,) and data type int64